In [ ]:
from pathlib import Path
import csv
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import soundfile as sf

In [ ]:
fmc = Path("./FakeMusicCaps/")
data_sets = [
    "MusicCaps",
    "MusicGen_medium",
    "SunoCaps",
    "audioldm2",
    "musicldm",
    "mustango",
    "stable_audio_open",
]

In [ ]:
import os

for ds in data_sets:
    print(ds)
    print(len(os.listdir(f'./FakeMusicCaps/{ds}')))
    print()

In [ ]:
y, sr = librosa.load(fmc / "SunoCaps/1.wav")

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.waveshow(y, sr=sr)
plt.show()
plt.close()

In [ ]:

print(sr)
print(y)

In [ ]:
# Viewing waveforms
for folder in data_sets:
    folder_path = fmc / folder
    for music in folder_path.glob('*.wav'):
        y, sr = librosa.load(music)
        plt.figure(figsize=(14, 5))
        librosa.display.waveshow(y, sr=sr)
        plt.show()
        break

In [ ]:
# For viewing spectrograms
for folder in data_sets:
    folder_path = fmc / folder
    for music in folder_path.glob('*.wav'):
        y, sr = librosa.load(music)
        mel_signal = librosa.feature.melspectrogram(y=y, sr=sr)
        spectrogram = np.abs(mel_signal)
        power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
        plt.figure(figsize=(8, 7))
        librosa.display.specshow(power_to_db, sr=sr, x_axis='time', y_axis='mel', cmap='magma')
        plt.colorbar(label='dB')
        plt.title('Mel-Spectrogram (dB)', fontdict=dict(size=18))
        plt.xlabel('Time', fontdict=dict(size=15))
        plt.ylabel('Frequency', fontdict=dict(size=15))
        plt.show()
        break

In [ ]:
modified_fake_music_caps_folder = Path('./PitchShiftedFakeMusicCaps')
modified_fake_music_caps_folder.mkdir(exist_ok=True)

for folder in data_sets:
    folder_path = fmc / folder
    print(f"Writing to {modified_fake_music_caps_folder / folder}")
    for song in folder_path.glob('*.wav'):
        destination_folder = Path(modified_fake_music_caps_folder / folder)
        destination_folder.mkdir(exist_ok=True)
        y, sr = librosa.load(song)
        low, high = -2, 2
        steps = float((high - low) * np.random.rand() + low)
        new_y = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=steps)
        print(f"\tnew song {song.stem}.wav")
        sf.write(destination_folder / f"{song.stem}.wav", new_y, sr, )

In [ ]:
# Create and save spectrograms

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib
matplotlib.use('agg')

spectrogram_folder = Path('./Spectrograms/')
spectrogram_folder.mkdir(exist_ok=True)

for folder in data_sets:
    folder_path = fmc / folder
    for music in folder_path.glob('*.wav'):
        destination_folder = Path(spectrogram_folder / folder)
        destination_folder.mkdir(exist_ok=True)
        y, sr = librosa.load(music)
        mel_signal = librosa.feature.melspectrogram(y=y, sr=sr)
        spectrogram = np.abs(mel_signal)
        power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
        fig = plt.figure(figsize=(8, 7))
        canvas = FigureCanvas(fig)
        p = librosa.display.specshow(power_to_db, sr=sr, x_axis='time', y_axis='mel', cmap='magma')
        fig.savefig(destination_folder / f"{music.stem}.png")
        plt.close()

In [ ]:
import gc
gc.collect()